# _**DatathONS - 4ª Edição**_
## **Equipe**:
* ### _Thais_
* ### _Bernardo_
* ### _Luiz Flavio Pereira_
* ### _Marcelo[texto do link](https://)_


_________________________
_________________________
O desafio desta 4ª edição é sobre o uso de inteligência para validação de dados de contratos de conexão de transmissão. A solução deverá ser composta de modelo com técnicas de Machine Learning e Natural Language Processing.<br><br>

**SOBRE A SOLUÇÃO**: Buscamos uma solução que possa realizar uma análise prévia, indicando se aquele documento está ou não conforme para assinatura, uma vez que a maioria das informações que precisam ser validadas encontram-se em base de dados do ONS (histórico de documentos de um mesmo agente) ou até mesmo em bases públicas (ANEEL e Receita Federal). Esperamos que essa triagem reduza a quantidade de contratos que são analisados manualmente e acelerem o processo de assinatura

________________________
## _**Importação dos arquivos através do Google Drive**_

Importação dos arquivos para o ambiente do Google Colab através do Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Definicao do caminho da pasta contendo os arquivos no Google Drive

drive_path = '/content/drive/MyDrive/DatathONS/'

_________________
## _**Setup do Programa**_

Instalação e importação das bibliotecas e descompatação dos arquivos

In [3]:
# Remove avisos de "warning"

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Instala a biblioteca tika (Realiza a leitura dos arquivos em pdf)
!pip install tika

# Instala a biblioteca unidecode (Permite a remocao de caracteres acentuados e especiais)
!pip install unidecode

# Instala a biblioteca fuzzywuzzy (Biblioteca que permite a manipulacao e comparacao de strings)
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Importa as bibliotecas essenciais ao desenvolvimento do programa

import os
import re
import fuzzywuzzy
import pandas as pd
from datetime import datetime, timedelta, date

from zipfile import ZipFile
from unidecode import unidecode
from tika import parser
from fuzzywuzzy import fuzz, process

In [6]:
# Define o diretorio para leitura dos arquivos

os.chdir(drive_path)
os.listdir()

['DatathONS.zip',
 'Dados de Agentes e Outorgas.zip',
 'Parecer de Acesso.zip',
 'Contratos_com_problemas.zip',
 'Contratos_sem_problemas.zip']

In [7]:
# Lista contendo o nome dos arquivos para iteracao e descompactacao

zip_files = os.listdir()
zip_files

['DatathONS.zip',
 'Dados de Agentes e Outorgas.zip',
 'Parecer de Acesso.zip',
 'Contratos_com_problemas.zip',
 'Contratos_sem_problemas.zip']

In [8]:
# Descompactacao dos arquivos no diretorio raiz do Google Colab

os.chdir('/content/')

for file_ in zip_files:
    path = drive_path + file_
    
    with ZipFile(path, mode="r") as archive:
        archive.extractall()
    
    print(f'O arquivo {file_} foi descompactado.')

O arquivo DatathONS.zip foi descompactado.
O arquivo Dados de Agentes e Outorgas.zip foi descompactado.
O arquivo Parecer de Acesso.zip foi descompactado.
O arquivo Contratos_com_problemas.zip foi descompactado.
O arquivo Contratos_sem_problemas.zip foi descompactado.


In [9]:
# Define o caminho dos contratos a serem lidos e tratados no programa

os.chdir('/content/CCTs assinados digitalmente')
ctt_ok = os.listdir()
print(ctt_ok)

['CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf', 'CCT-2022-166-00 - CHESF e UFV ZEBU II (0132022)-Assinado.pdf', 'CCT-2019-118-01 - SANTA LUZIA e CHAFARIZ 6-Assinado.pdf', 'CCT-2021-193-00 - Esperanza x EOL São Ricardo 06-Assinado.pdf', 'CCT-2016-091-01 IE PINHEIRO - ELEKTRO-Assinado.pdf', 'CCT-2021-276-00 - CHESF e UFV Pitombeira-Assinado.pdf', 'CCT-2021-204-00 - CHESF e UFV CIRANDA 3 (039-2021)-Assinado.pdf', 'CCT-2022-347-00 - SMTE x UFV BOA SORTE 08 (008-2022)-Assinado.pdf', 'CCT-2020-159-01 - CHESF e  UFV MAURITI 1-Assinado.pdf', 'CCT-2014-025-02 LNT e CELG-Assinado.pdf', 'CCT-2020-101-000 - CHESF-UFV Juazeiro V (014-2020)-Assinado.pdf', 'CCT-2022-105-01 - ESPERANZA x UFV SERRA DO MEL VI-Assinado.pdf', 'CCT-2020-291-01 - SOLARIS x JAÍBA L2-Assinado.pdf', 'CCT-2000-019-48 CTEEP e ELEKTRO-Assinado.pdf', 'CCT-2022-100-00 - SITE e UFV MARANGATU 6-Assinado.pdf', 'CCT-2015-121-04 - CGT ELETROSUL e CELESC-Assinado.pdf', 'CCT-2020-296-00 - SOLARIS e JAÍBA SE 2-

In [10]:
# Geracao do DataFrame contendo dados dos agentes

os.chdir('/content/')
df_agentes = pd.read_csv('/content/Base de dados Agentes e Outorgas.csv', dtype='object', sep=';')
df_agentes.head(2)

,Id Agente,Sgl Agente,Dsc Razao Social,Dsc Cnpj,Dsc Logradouro,Dsc Numero,Dsc Complemento,Dsc Bairro,Nome,Estad Id,Nom Reduzido,Cod Regulamentacao,Dsc Titulo,Dat Emissao,Dat Publicacao,Dsc Ementa
0,162,RGE SUL,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,02016440000162,Rua: Dona Laura,320,14º andar,Rio Branco,Porto Alegre,RS,RGE SUL,CCO-1997-012,Contrato de Concessão de Distribuição nº 1997-012,01/01/1900,NaN,Distribuidora - AES Sul Distribuidora Gaúcha d...
1,162,RGE SUL,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,02016440000162,Rua: Dona Laura,320,14º andar,Rio Branco,Porto Alegre,RS,RGE SUL,DSP-2007-3741,Despacho - Nº 3741/2007,27/12/2007,28/12/2007,Anui com a proposta de alteração do art. 5º do...


_____________
_____________
# **_Grupo 1: Extração e validação de informações do dados das instituições envolvidas e Grupo 2: Extração e validação das informações de outorga_**

________________________________
## _**Funções para Leitura, Busca das informações e Comparação dos Dados dos Contratos e Base de Dados**_

As funções abaixo propiciam uma forma prática de se capturar as informações no contrato e compará-las no banco de dados dos agentes.

In [11]:
def ctt_read(ctt):
    '''
    Funcao para leitura do arquivo pdf com os dados do contrato.

    Retorna o contrato em formato de uma unica string sem quebras de linha, 
    apenas com letras minusculas, sem carecteres acentuados e especiais e espacos duplos.
    '''

    ctt_path = 'CCTs assinados digitalmente/' + ctt
    raw_ctt = parser.from_file(ctt_path)
    ctt_limpo = unidecode(re.sub('\n', '', raw_ctt['content']).lower())
    ctt_limpo = re.sub('  ', ' ', ctt_limpo)

    return ctt_limpo

In [12]:
print(ctt_read.__doc__)


    Funcao para leitura do arquivo pdf com os dados do contrato.

    Retorna o contrato em formato de uma unica string sem quebras de linha, 
    apenas com letras minusculas, sem carecteres acentuados e especiais e espacos duplos.
    


In [13]:
def cnpj_read(ctt):
    '''    Funcao para leitura do arquivo pdf com os dados do CNPJ constante no contrato.
    
    Atraves da mascara do CNPJ, o programa se utiliza de expressoes regulares para encontrar as ocorrencias no contratos
    Retorna um DataFrame contendo os CNPJs encontrados no contrato em formato de uma unica string numerica (cada CNPJ)
    O formato de DataFrame foi escolhido para possibilitar um join com a tabela de agentes, retornando apenas os dados de interesse
    '''

    cnpj_mask = "\d+\.*?\.\d+\.*?\.\d+\.*?\/\d+\.*?\-\d+"
    cnpjs_list = []
    cnpjs = set(re.findall(cnpj_mask, ctt))

    for cnpj in cnpjs:
        cnpjs_list.append(re.sub('[./-]', '', cnpj))

    df = pd.DataFrame(cnpjs_list, columns=['cnpj'])

    return df

In [14]:
def extrai_agentes(df_cnpjs):

    '''    Funcao para extracao dos dados dos agentes com CNPJ existente no contrato.

    A funcao realiza um inner join do dataframe contendo os CNPJs encontrados no contrato.
    Em seguida realiza a limpeza e tratamento dos dados e retorna um DataFrame apenas com os CNPJs existentes
    no contrato e na base de dados.
    '''

    df = df_cnpjs.merge(df_agentes, how='inner', left_on='cnpj', right_on='Dsc Cnpj')
    df.fillna('', inplace=True)
    df.drop_duplicates(subset=['Dsc Cnpj'], inplace=True)
    df.drop('cnpj', inplace=True, axis=1)

    for column in df.columns:
        df[column] = df.apply(lambda x: x[column].lower(), axis=1)
        df[column] = df.apply(lambda x: unidecode(x[column]), axis=1)
    return df

In [15]:
def cnpj_test(cnpj, agentes):

    '''    Funcao para conforntacao dos CNPJs existentes no contrato e na base de dados.

    A funcao realiza um left join do dataframe contendo os CNPJs encontrados no contrato com o DataFrame existente na base de dados,
    removendo as ocorrencias duplicadas existentes na base de agentes.
    Em seguida realiza a limpeza e tratamento dos dados e retorna um DataFrame apenas com os CNPJs existentes
    no contrato e na base de dados.

    Retorna uma lista com os CNPJs existentes no contrato mas INEXISTENTES na base de agentes, o que caracteriza uma condicao
    de faha no contrato (CNPJ incorreto ou inexistente).
    '''

    df = df_cnpjs.merge(df_agentes, how='left', left_on='cnpj', right_on='Dsc Cnpj')
    df_cnpj_ausentes = df[df['Dsc Razao Social'].isna()]
    df.drop_duplicates(subset=['Dsc Cnpj'], inplace=True)
     
    return list(df_cnpj_ausentes['cnpj'])

In [16]:
def razao_social_test(df_agentes_ctt, contrato):

    '''    Funcao para conforntacao das Razoes Sociais existentes no contrato e na base de dados.

    A funcao extrai a razao social de cada agente importado da base de dados de agentes (CNPJ existente no contrato e na base)
    e executa uma busca de similaridade na string do contrato. Quanto mais alto o score retornado, mais similaridade e possbilidade
    de existencia do nome da razao social no texto do contrato.
    '''

    similaridade_razao_social = set()
    razao_social = list(df_agentes_ctt['Dsc Razao Social'])

    for razao in razao_social:
        razao = re.sub('[.-]', '', razao)
        razao = re.sub('  ', ' ', razao).strip().lstrip()
        razao_social_ctt = re.sub('[.-]', '', contrato)
        similaridade_razao_social.add((razao.title(), fuzz.token_set_ratio(razao, razao_social_ctt)))

    return similaridade_razao_social

In [17]:
def endereco_test(df_agentes_ctt, contrato):

    '''    Funcao para conforntacao dos enderecos existentes no contrato e na base de dados.

    A funcao extrai os enderecos de cada agente importado da base de dados de agentes (CNPJ existente no contrato e na base)
    e executa uma busca de similaridade na string do contrato. Quanto mais alto o score retornado, mais similaridade e possibilidade
    de existencia do endereco no texto do contrato.
    '''

    similaridade_endereco = set()
    df_agentes.fillna('', inplace=True)
    enderecos = list(df_agentes_ctt.apply(lambda x: x['Dsc Logradouro'] + ' ' + x['Dsc Numero'] + ' ' + x['Dsc Complemento'] + ' ' + x['Dsc Bairro'], axis=1))

    for endereco in enderecos:
        endereco = re.sub('[,.-]', '', endereco)
        endereco = unidecode(re.sub('  ', ' ', endereco).strip().lstrip())
        endereco_ctt = re.sub('[.-]', '', contrato)
        similaridade_endereco.add((endereco.title(), fuzz.token_set_ratio(endereco, endereco_ctt)))

    return similaridade_endereco

____________________________
## _**Análise dos Contratos**_

O código abaixo realiza a análise dos contratos com a utilização das funções criadas acima.

In [18]:
# Seleciona apenas alguns contratos dentre todos os importados da base de arquivos de contrato.
# (Utilizada para desenvolvimento do programa)

ctt_ok_teste = ctt_ok[0:20]
ctt_ok_teste

['CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf',
 'CCT-2022-166-00 - CHESF e UFV ZEBU II (0132022)-Assinado.pdf',
 'CCT-2019-118-01 - SANTA LUZIA e CHAFARIZ 6-Assinado.pdf',
 'CCT-2021-193-00 - Esperanza x EOL São Ricardo 06-Assinado.pdf',
 'CCT-2016-091-01 IE PINHEIRO - ELEKTRO-Assinado.pdf',
 'CCT-2021-276-00 - CHESF e UFV Pitombeira-Assinado.pdf',
 'CCT-2021-204-00 - CHESF e UFV CIRANDA 3 (039-2021)-Assinado.pdf',
 'CCT-2022-347-00 - SMTE x UFV BOA SORTE 08 (008-2022)-Assinado.pdf',
 'CCT-2020-159-01 - CHESF e  UFV MAURITI 1-Assinado.pdf',
 'CCT-2014-025-02 LNT e CELG-Assinado.pdf',
 'CCT-2020-101-000 - CHESF-UFV Juazeiro V (014-2020)-Assinado.pdf',
 'CCT-2022-105-01 - ESPERANZA x UFV SERRA DO MEL VI-Assinado.pdf',
 'CCT-2020-291-01 - SOLARIS x JAÍBA L2-Assinado.pdf',
 'CCT-2000-019-48 CTEEP e ELEKTRO-Assinado.pdf',
 'CCT-2022-100-00 - SITE e UFV MARANGATU 6-Assinado.pdf',
 'CCT-2015-121-04 - CGT ELETROSUL e CELESC-Assinado.pdf',
 'CCT-2020-296-00 - SOLAR

In [19]:
# Busca cada contrato individual dentro da lista de contratos e aplica as funcoes definidas acima

for ctt in ctt_ok_teste:
    contrato = ctt_read(ctt)
    df_cnpjs = cnpj_read(contrato)
    df_agentes_ctt = extrai_agentes(df_cnpjs)

    print('\n\n-----------------------------------------------------------------------------------------------------------------------------------------')
    print(f'INFORMACOES DO CONTRATO: {ctt}')
    print('-----------------------------------------------------------------------------------------------------------------------------------------\n')

    ausentes = cnpj_test(df_cnpjs, df_agentes)
    print(f'OS CNPJS CITADOS NO CONTRATO E DESCRITOS ABAIXO NAO FORAM ENCONTRADOS NA BASE DE DADOS DOS AGENTES:\n{ausentes}')

    razao_social = razao_social_test(df_agentes_ctt, contrato)
    print(f'\n\nAS RAZOES SOCIAIS CITADAS NO CONTRATO POSSUEM A SIMILARIDADE DEMONSTRADA ABAIXO QUANDO COMPARADA COM A BASE DE DADOS DOS AGENTES:')
    print('(Nome da empresa citada no contrato, Percentual de compatibilidade)\n')

    for razao in razao_social:
        print(razao)

    enderecos = endereco_test(df_agentes_ctt, contrato)
    print(f'\n\nOS ENDERECOS CITADOS NO CONTRATO POSSUEM A SIMILARIDADE DEMONSTRADA ABAIXO QUANDO COMPARADA COM A BASE DE DADOS DOS AGENTES:')
    print('(Endereco da empresa citada no contrato, Percentual de compatibilidade)\n')

    for endereco in enderecos:
        print(endereco)



-----------------------------------------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------------------------------------

OS CNPJS CITADOS NO CONTRATO E DESCRITOS ABAIXO NAO FORAM ENCONTRADOS NA BASE DE DADOS DOS AGENTES:
['41489453000100', '41489475000170', '41484020000162', '41486632000194', '41413101000171', '41503859000109', '41489491000163', '02831210000157', '41489523000120', '41483847000151', '41483602000124']


AS RAZOES SOCIAIS CITADAS NO CONTRATO POSSUEM A SIMILARIDADE DEMONSTRADA ABAIXO QUANDO COMPARADA COM A BASE DE DADOS DOS AGENTES:
(Nome da empresa citada no contrato, Percentual de compatibilidade)

('Etb Empresa De Transmissao Baiana Sa', 100)


OS ENDERECOS CITADOS NO CONTRATO POSSUEM A SIMILARIDADE DEMONSTRADA ABAIXO 

____________________
____________________
# **_Grupo 3: Extração e validação das informações do Parecer de Acesso_**

______________
## _**Detalhamento do Parecer de Acesso do Contrato**_

O código abaixo busca o parecer no contrato e o compara com a base de dados dos pareceres fornceida.

In [20]:
# Realiza a importacao dos dados da base de dados dos pareceres e a armazena em formato de Pandas DataFrame

df_parecer_acesso = pd.read_excel('/content/Parecer de acesso.xlsx')
df_parecer_acesso.head()

,documento_emitido,data_emissao,pontoconexao
0,2020-PA-0030-R0-ra,2020-03-11 07:03:06.913,São Mateus do Sul 230 kv
1,2021-PA-0009-R0-ra,2021-01-13 12:45:45.570,SE Porto Velho 230 kv
2,2018-PA-0005-RA,2018-08-28 11:08:45.693,NaN
3,2022-PA-0024-R0-ra,2022-03-10 16:35:53.950,Curral Novo do Piauí II 230 kv
4,2019-PA-0076-R2,2019-04-24 12:02:53.717,João Câmara II 230 kv


In [21]:
# Realiza a transformacao dos caracteres alfabeticos da coluna com o numero do documento para caracteres minusculos
# Realiza a conversao do formato timestamp da data de emissao para o formato de data

df_parecer_acesso['documento_emitido'] = df_parecer_acesso.apply(lambda x: x['documento_emitido'].lower(), axis=1)
df_parecer_acesso['data_emissao'] = df_parecer_acesso.apply(lambda x: x['data_emissao'].date(), axis=1)
df_parecer_acesso.head()

,documento_emitido,data_emissao,pontoconexao
0,2020-pa-0030-r0-ra,2020-03-11,São Mateus do Sul 230 kv
1,2021-pa-0009-r0-ra,2021-01-13,SE Porto Velho 230 kv
2,2018-pa-0005-ra,2018-08-28,NaN
3,2022-pa-0024-r0-ra,2022-03-10,Curral Novo do Piauí II 230 kv
4,2019-pa-0076-r2,2019-04-24,João Câmara II 230 kv


In [22]:
def possui_parecer(df_parecer_acesso, contrato):

    '''    Funcao para busca do parecer de acesso no contrato e comparacao com a base de dados.

    A funcao busca o termo "parecer de acesso" no contrato (modus operandi comum nos contratos com esse parecer emitido).
    Se o termo for encontrado, faz-se uma busca pelo numero atraves das mascaras para o codigo do documento, sendo estes salvos em um DataFrame.
    Eh realizado um inner join entre esse DataFrame e o DataFrame da base de pareceres, sendo retornado apenas aquele comuns aos dois.

    A data atual eh obtida para a realizacao da diferenca de dias entre a liberacao do parecer e a data de hoje, sendo esta diferenca
    categorizada entre tres rotulos possiveis.
    '''

    if re.search('parecer de acesso', contrato):
        print('Existe mencao ao parecer de acesso no contrato, com o(s) codigo(s) exibido(s) abaixo:')
        if re.findall(r"[\d\.]+-pa+-[\d\.]+-[\w\.]+-ra+", contrato) == []:
            parecer = set(re.findall(r"[\d\.]+-pa+-[\d\.]+-[\w\.]+", contrato))
        else:
            parecer = set(re.findall(r"[\d\.]+-pa+-[\d\.]+-[\w\.]+-ra+", contrato))

        print(parecer)

        print('-------------------------------------------------------------')
        df = pd.DataFrame(parecer, columns=['parecer'])

        df_parecer = df.merge(df_parecer_acesso, how='inner', left_on='parecer', right_on='documento_emitido')
        df_parecer.drop('parecer', inplace=True, axis=1)

        hoje = date.today()

        df_parecer['dias_delay'] = df_parecer.apply(lambda x: hoje - x['data_emissao'], axis=1)
        df_parecer['resultado'] = df_parecer.apply(lambda x: 'Parecer vencido' if x['dias_delay'] > timedelta(days=90) else\
                                                             ('Vencimento em menos de 30 dias' if x['dias_delay'] > timedelta(days=60) else 'Status ok'), axis=1)

        df_parecer['dias_delay'] = df_parecer.apply(lambda x: str(x['dias_delay']), axis=1)

        for k in df_parecer.iterrows():
            #print(k[1]['dias_delay'].split()[0])
            print(f"O parecer de acesso {k[1]['documento_emitido'].upper()} datado de {k[1]['data_emissao']}")
            print(f"possui um atraso de {k[1]['dias_delay'].split()[0]} dias, com status de {k[1]['resultado'].upper()}.\n")

    else:
        print('Nao existe mencao ao parecer de acesso no contrato.')

In [23]:
# Busca cada contrato individual dentro da lista de contratos e aplica a funcao definida acima

for ctt in ctt_ok_teste:
    contrato = ctt_read(ctt)
    df_cnpjs = cnpj_read(contrato)
    df_agentes_ctt = extrai_agentes(df_cnpjs)

    print('\n\n----------------------------------------------------------------------------------------------------------')
    print(f'INFORMACOES DO CONTRATO: {ctt}')
    print('-----------------------------------------------------------------------------------------------------------')
    possui_parecer(df_parecer_acesso, contrato)



----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
Existe mencao ao parecer de acesso no contrato, com o(s) codigo(s) exibido(s) abaixo:
{'2022-pa-0068-r0', '2022-pa-0068-r0junho'}
-------------------------------------------------------------
O parecer de acesso 2022-PA-0068-R0 datado de 2022-06-14
possui um atraso de 131 dias, com status de PARECER VENCIDO.



----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-166-00 - CHESF e UFV ZEBU II (0132022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
Existe mencao ao parecer de acesso no contrato, com o(s) codigo(s) exib

____________________
____________________
# **_Grupo 4: Qualificação dos diretores_**

______________
## _**Qualificação dos diretores do ONS**_

Esta etapa busca a verificação da citação dos nomes do diretores e se seus cargos estão corretamente citados no contrato.

In [24]:
# Esse trecho de codigo confronta um texto padrao e experado existir nos contratos com sua ocorrencia no contrato.
# Ainda, os nomes corretos dos cargos sao confrontados no contrato para sua verificacao.
# Para os dois casos eh retornado um score com a pontuacao de similaridade, que quanto maior, maior a 
# probabilidade da ocorrencia do trecho no contrato. 

texto = "Neste ato representado  por seus representantes legais, ao final assinados, doravante denominado ONS"
texto = re.sub('[,.-]', '', texto).lower()
texto = unidecode(re.sub('  ', ' ', texto).strip().lstrip())

diretor_geral = "diretor geral"
diretor_ti = "diretor de ti"
relacionamento = "relacionamento com agentes e assuntos regulatorios"

for ctt in ctt_ok_teste:
    contrato = ctt_read(ctt)
    texto_ctt = re.sub('[.-]', '', contrato)
    texto_ctt = re.sub('  ', ' ', contrato)

    print('\n\n----------------------------------------------------------------------------------------------------------')
    print(f'INFORMACOES DO CONTRATO: {ctt}')
    print('-----------------------------------------------------------------------------------------------------------')
    print(f'SIMILARIDADE DO TEXTO PADRAO NO CONTRATO: {fuzz.token_set_ratio(texto, texto_ctt)}%')
    print(f'SIMILARIDADE DO CARGO DE DIRETOR GERAL CITADO: {fuzz.token_set_ratio(diretor_geral, texto_ctt)}%')
    print(f'SIMILARIDADE DO CARGO DE DIRETOR DE TI CITADO: {fuzz.token_set_ratio(diretor_ti, texto_ctt)}%')
    print(f'SIMILARIDADE DO CARGO DE RELACIONAMENTO COM AGENTES CITADO: {fuzz.token_set_ratio(relacionamento, texto_ctt)}%') 



----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
SIMILARIDADE DO TEXTO PADRAO NO CONTRATO: 100%
SIMILARIDADE DO CARGO DE DIRETOR GERAL CITADO: 100%
SIMILARIDADE DO CARGO DE DIRETOR DE TI CITADO: 100%
SIMILARIDADE DO CARGO DE RELACIONAMENTO COM AGENTES CITADO: 100%


----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-166-00 - CHESF e UFV ZEBU II (0132022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
SIMILARIDADE DO TEXTO PADRAO NO CONTRATO: 100%
SIMILARIDADE DO CARGO DE DIRETOR GERAL CITADO: 100%
SIMILARIDADE DO CARGO DE DIRETOR DE TI CITADO: 87%
SIMILARIDADE DO 

____________________
____________________
# **_Grupo 5: Definição de Procedimentos de Rede_**

_____________
## _**Definição correta dos Procedimentos de Rede**_

Esta etapa busca a verificação da correta citação das definições do Procedimento de Rede nos contratos enviados ao ONS.

In [25]:
# Esse trecho de codigo confronta um texto padrao de citacao dos Procedimentos de Rede e experado existir nos 
# contratos com sua ocorrencia nos mesmos.
# Eh retornado um score com a pontuacao de similaridade, que quanto maior, maior a probabilidade da ocorrencia do trecho no contrato. 

texto_proc_rede = "Documento elaborado pelo ONS, com a participação dos agentes e aprovado pela ANEEL, que estabelece os procedimentos e os \
                   requisitos técnicos para o planejamento, a IMPLANTAÇÃO, o uso e a operação do SISTEMA DE TRANSMISSÃO, bem como as responsabilidades do ONS e das PARTES;"
texto_proc_rede = re.sub('[;,.-]', '', texto_proc_rede).lower()
texto_proc_rede = unidecode(re.sub('  ', ' ', texto_proc_rede).strip().lstrip())

for ctt in ctt_ok_teste:
    contrato = ctt_read(ctt)
    texto_ctt = re.sub('[.-]', '', contrato)
    texto_ctt = re.sub('  ', ' ', contrato)

    print('\n\n----------------------------------------------------------------------------------------------------------')
    print(f'INFORMACOES DO CONTRATO: {ctt}')
    print('-----------------------------------------------------------------------------------------------------------')
    print(f'SIMILARIDADE DO TEXTO DE DEFINICAO DE PROCEDIMENTO DE REDE NO CONTRATO: {fuzz.token_set_ratio(texto_proc_rede, texto_ctt)}%')



----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-364-00 - ETB x VENTOS DE SANTA LUZIA 08 (159-2022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
SIMILARIDADE DO TEXTO DE DEFINICAO DE PROCEDIMENTO DE REDE NO CONTRATO: 100%


----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2022-166-00 - CHESF e UFV ZEBU II (0132022)-Assinado.pdf
-----------------------------------------------------------------------------------------------------------
SIMILARIDADE DO TEXTO DE DEFINICAO DE PROCEDIMENTO DE REDE NO CONTRATO: 100%


----------------------------------------------------------------------------------------------------------
INFORMACOES DO CONTRATO: CCT-2019-118-01 - SANTA LUZIA e CHAFARIZ 6-Assinado.pdf
--------------------------------------